# Explanation of symbolic derivatives using `sympy`

---

I've been listening to a lot of anguish over this assignment, so I thought I'd investigate sympy for everyone's sake.
I like to keep most things under some kind of namespace, so that anyone new looking at the code will have some clue which package these functions are coming from.

In [1]:
import sympy as sp

Let's start with the basics from the `sympy` documentation. Mathematica treats every variable as a symbol, but for Python, we'll have to explicitly tell it when we want to use symbolic variables.

In [8]:
# Create our base set of variables
symbols = sp.symbols
x, y, z, t = symbols('x y z t')

Now that we have some base symbolic variables, we can build up more complicated equations. Just like we have `numpy.sin` for array calculations, we've got `sympy.sin` for symbolic ones.

In [10]:
# Create a function
loglike = sp.sin(x * y)


That's nice and all, but we really need to be able to do derivatives for the homework.
> Note: I'm fairly sure the rendering of sympy variables is system dependent. I've got TeXLive installed, so I'm getting LaTeX out of this.

In [11]:
# Calculate the partial derivatives
sp.derive_by_array(loglike, [x, y, z])


[y*cos(x*y), x*cos(x*y), 0]

<br>
<br>

## Jacobian's - Pretty sure it's related to the $A$ matrix.
There's some debate on the internet if the Jacobian should be displayed as we're used to, or transposed. We'll investigate that using the first example from Wikipedia's Jacobian page.

In [33]:
func = [x**2 * y, 5*x + sp.sin(y)]
sp.derive_by_array(func, [x, y])



[[2*x*y, 5], [x**2, cos(y)]]

Look's like tensor derivatives yield the transposed Jacobian.

In [32]:
sp.derive_by_array(func, [x, y]).transpose() # .T is for numpy, not sympy

[[2*x*y, x**2], [5, cos(y)]]

Much better. Now let's check it against `sympy`'s Jacobian example within the `sympy.Matrix` module.

In [21]:
X, rho, phi = symbols('X rho phi')
X = [rho * sp.cos(phi), rho * sp.sin(phi), rho**2]
sp.derive_by_array(X, [rho, phi]).transpose()

[[cos(phi), -rho*sin(phi)], [sin(phi), rho*cos(phi)], [2*rho, 0]]

In [22]:
from sympy import Matrix
X = Matrix(X)
X.jacobian([rho, phi])

Matrix([
[cos(phi), -rho*sin(phi)],
[sin(phi),  rho*cos(phi)],
[   2*rho,             0]])

<br>
<br>

## Hessian's

Those are matching. Now, if we run the derivative twice, will we get a Hessian? Or, will we need to transpose it at the end? Let's:

* Check the formula for a Hessian using generic functions, so we can check for typos
* Use realized symbolic functions to do the same
* Check if `derive_by_array` matches

### Checking the formula

In [24]:
f = sp.Function('f')(x, y) # Define generic functions
sp.hessian(f, (x, y))



Matrix([
[Derivative(f(x, y), (x, 2)),   Derivative(f(x, y), x, y)],
[  Derivative(f(x, y), x, y), Derivative(f(x, y), (y, 2))]])

That look's good. We've got the $\partial x_i^2$'s in the diagonal and the $\partial y \partial x$ is running down the first column.

### Realized symbolic functions

In [27]:
f = x**3 + 3 * x * y**5
f

x**3 + 3*x*y**5

In [29]:
sp.hessian(f, (x, y))

Matrix([
[    6*x,   15*y**4],
[15*y**4, 60*x*y**3]])

Matches up with what we would expect from the generic function above.

### Check the `derive_by_array` result
We can start with the gradient.

In [30]:
# Calculate a Hessian using the derive by array function
grad = sp.derive_by_array(f, [x, y])
grad


[3*x**2 + 3*y**5, 15*x*y**4]

There's probably a way to nest these functions together, but at least we can check the intermediate calculations this way.

In [31]:
hess = sp.derive_by_array(grad, [x, y])
hess


[[6*x, 15*y**4], [15*y**4, 60*x*y**3]]

Look's the same as the above. So I suppose there might be fewer programming errors if we chose the transposed Jacobian.

<br>
<br>

## Substitution of real numbers into our equations.
Let's do some basic substitutions to look at.
> Note: Almost everything in `sympy` is an immutable data type. So if we want to use something in a later calculation, we should assign it to a new variable.

In [34]:
hess.subs(x, 1)

[[6, 15*y**4], [15*y**4, 60*y**3]]

In [38]:
hess.subs(y, 1)

[[6*x, 15], [15, 60*x]]

Those make sense to me. Let's do both at once:
> Note: It is **very** easy to miss that the tuples are contained in a list. You'll be left with a symbol within your loop.

In [39]:
hess.subs([(x, 1), (y, 2)])

[[6, 240], [240, 480]]

Now, how to we get those values out of there? Let's check the type and try the usual.

In [49]:
a = hess.subs([(x, 1), (y, 2)])
type(a)

sympy.tensor.array.dense_ndim_array.ImmutableDenseNDimArray

In [50]:
a[0,1]

240

Works as expected. Maybe we want to get them all at once?

In [54]:
a.tolist()

[[6, 240], [240, 480]]

### Speeding things up!!!
Symbolic calculations are **slow**. We can use `sympy.lambdify` to speed things back
up to `numpy` speeds. Lambda functions are python's implementation of
*anonymous* functions. These are bite sized functions that cannot live
outside of their file. If we needed to use their functionality somewhere
else, we would have to promote them to real functions and save them to a
module file.

> Note: This is what you want to put in your loops

In [58]:
from sympy.utilities import lambdify
fast_func = lambdify((x, y), hess, 'numpy')
fast_func(1, 2)

[[6, 240], [240, 480]]

In [60]:
fast_func(1, 2) == a

False

In [61]:
fast_func(1, 2) == a.tolist()

True

Excellent! Now I'm ready to start the assignment painfully late in the game!!!

<br>
<br>
<br>
> Note: Below is the example from `sympy` when you look up their hessian function. They basically just refer you to the wiki, but I wouldn't put it past Dr. Qu to throw some constrained optimization at us, so I left it in here.

In [7]:
# Calculate a Hessian (bordered hessian for optimization problems)
f = sp.Function('f')(x, y)
g1 = sp.Function('g')(x, y)
g2 = x**2 + 3 * y
sp.hessian(f, (x, y), [g1, g2])



Matrix([
[                     0,   0,      Derivative(g(x, y), x),      Derivative(g(x, y), y)],
[                     0,   0,                         2*x,                           3],
[Derivative(g(x, y), x), 2*x, Derivative(f(x, y), (x, 2)),   Derivative(f(x, y), x, y)],
[Derivative(g(x, y), y),   3,   Derivative(f(x, y), x, y), Derivative(f(x, y), (y, 2))]])